<a href="https://colab.research.google.com/github/jiayiwang5/Google-Colab/blob/master/chatbot_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pickle
import operator
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
data_dir = np.arange(0, 3000, 1000)
main_path = "/content/drive/My Drive/Colab Notebooks/"
def get_file_list(file_path):
    dir_list = os.listdir(file_path)
    if not dir_list:
        return
    else:
        dir_list = sorted(dir_list, key=lambda x: os.path.getmtime(os.path.join(file_path, x)))
    return dir_list
with open(main_path + 'middle_data/dictionary.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
for i, j in word_to_index.items():
      word_to_index[i] = j + 1
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key
with open(main_path + 'middle_data/words.pkl', 'rb') as f:
    words = pickle.load(f)
maxLen = 20
vocab_size = len(word_to_index) + 1
print('word_to_vec_map: ', len(list(words)))
print('vocab_size: ', vocab_size)
with open(main_path + 'middle_data/embedding_matrix.pkl', 'rb') as f:
      embedding_matrix = pickle.load(f)
print(embedding_matrix.shape)

word_to_vec_map:  400000
vocab_size:  42905
(42905, 50)


In [2]:
from keras.preprocessing import sequence
def generate_train(batch_size):
    print('\n*********************************generate_train()*********************************')
    steps=0
    context_ = np.load(main_path + 'middle_data/context_indexes.npy')
    final_target_ = np.load(main_path + 'middle_data/target_indexes.npy')
    context_ = context_[:90000]
    final_target_ = final_target_[:90000]
    epoch_mark = 0
    while True:
        context = context_[steps:steps+batch_size]
        final_target = final_target_[steps:steps+batch_size]
        if epoch_mark == 0:
          for pos, i in enumerate(final_target):
              for pos_, j in enumerate(i):
                  i[pos_] = j + 1
              if(len(i) > maxLen):
                  final_target[pos] = i[:maxLen]

          for pos, i in enumerate(context):
              for pos_, j in enumerate(i):
                  i[pos_] = j + 1
              if(len(i) > maxLen):
                  context[pos] = i[:maxLen]
  #         print('\ncontext.shape: ', context.shape)
          outs = np.zeros([context.shape[0], maxLen, vocab_size], dtype='float32')
          for pos, i in enumerate(final_target):
              for pos_, j in enumerate(i):
                  if pos_ > 20:
                      print(i)
                  if pos_ > 0:
                      outs[pos, pos_-1, j] = 1 # one-hot
  #             if pos%1000 == 0 :
  #                 print('{} entries completed'.format(pos)) # format()填充{}，格式化输出
  #         print('\nouts.shape: ', outs.shape)
        final_target = sequence.pad_sequences(final_target, maxlen=maxLen,
                                        dtype='int32', padding='post', 
                                         truncating='post')
        context = sequence.pad_sequences(context, maxlen=maxLen,
                                   dtype='int32', padding='post',
                                   truncating='post')
        yield ([context, final_target], outs)
        steps += batch_size
        if steps == 90000:
            steps = 0
            epoch_mark += 1

Using TensorFlow backend.


In [0]:
from keras.preprocessing import sequence
def generate_test(batch_size):
    print('\n*********************************generate_test()*********************************')
    steps=0
    context_ = np.load(main_path + 'middle_data/context_indexes.npy')
    final_target_ = np.load(main_path + 'middle_data/target_indexes.npy')
    context_ = context_[90000:]
    final_target_ = final_target_[90000:]
    epoch_mark = 0
    while True:
        context = context_[steps:steps+batch_size]
        final_target = final_target_[steps:steps+batch_size]
        if epoch_mark == 0:
          for pos, i in enumerate(final_target):
              for pos_, j in enumerate(i):
                  i[pos_] = j + 1
              if(len(i) > maxLen):
                  final_target[pos] = i[:maxLen]

          for pos, i in enumerate(context):
              for pos_, j in enumerate(i):
                  i[pos_] = j + 1
              if(len(i) > maxLen):
                  context[pos] = i[:maxLen]
  #         print('\ncontext.shape: ', context.shape)
          outs = np.zeros([context.shape[0], maxLen, vocab_size], dtype='float32')
          for pos, i in enumerate(final_target):
              for pos_, j in enumerate(i):
                  if pos_ > 20:
                      print(i)
                  if pos_ > 0:
                      outs[pos, pos_-1, j] = 1 # one-hot
  #             if pos%1000 == 0 :
  #                 print('{} entries completed'.format(pos)) # format()填充{}，格式化输出
  #         print('\nouts.shape: ', outs.shape)
        final_target = sequence.pad_sequences(final_target, maxlen=maxLen,
                                        dtype='int32', padding='post', 
                                         truncating='post')
        context = sequence.pad_sequences(context, maxlen=maxLen,
                                   dtype='int32', padding='post',
                                   truncating='post')
        yield ([context, final_target], outs)
        steps += batch_size
        if steps == 10000:
            steps = 0
            epoch_mark += 1

In [0]:
from keras.layers import Embedding
from keras.layers import Input, Dense, LSTM, TimeDistributed, Bidirectional, Concatenate
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import RMSprop

embed_layer = Embedding(input_dim=vocab_size, output_dim=50, trainable=True)
embed_layer.build((None,))
embed_layer.set_weights([embedding_matrix])

LSTM_cell = Bidirectional(LSTM(512, return_sequences=True, return_state=True))
LSTM_decoder = LSTM(1024, return_sequences=True, return_state=True)
LSTM_propagate = LSTM(1024, return_state=True)

dense = TimeDistributed(Dense(vocab_size, activation='softmax'))

  #encoder输入 与 decoder输入
input_context = Input(shape=(None, ), dtype='int32', name='input_context')
input_target = Input(shape=(None, ), dtype='int32', name='input_target')

input_context_embed = embed_layer(input_context)
input_target_embed = embed_layer(input_target)

encoder_out, forward_h, forward_c, backward_h, backward_c = LSTM_cell(input_context_embed)
context_h = Concatenate()([forward_h, backward_h])
context_c = Concatenate()([forward_c, backward_c])

encoder_pro, pro_h, pro_c = LSTM_propagate(encoder_out, initial_state=[context_h, context_c])

decoder_lstm, _, _ = LSTM_decoder(input_target_embed, 
                                    initial_state=[pro_h, pro_c])

output = dense(decoder_lstm)

model = Model([input_context, input_target], output)

rmsprop = RMSprop(lr=0.005, rho=0.9, epsilon=None, decay=0.0)

model.compile(optimizer=rmsprop, loss='categorical_crossentropy', 
                metrics=['accuracy'])
#   model.summary() 

filepath = main_path + "modles/W2-" + "-{epoch:03d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath,
                             monitor='loss',
                             verbose=1,
                             save_best_only=False,
                             mode='min',
                             period=1,
                             save_weights_only=True
                             )
reduce_lr = ReduceLROnPlateau(monitor='loss', 
                              factor=0.2, 
                              patience=5, 
                              verbose=1, 
                              mode='min', 
                              min_delta=0.0001, 
                              cooldown=0, 
                              min_lr=0
                              )
callbacks_list = [checkpoint, reduce_lr]
  
initial_epoch=0
file_list = os.listdir(main_path + 'modles/')
if len(file_list) > 0:
  epoch_list = get_file_list(main_path + 'modles/')
  epoch_last = epoch_list[-1]
  model.load_weights(main_path + 'modles/' + epoch_last)
#     if len(file_list) > 2:
#         for file_name in file_list[:-2]:
#             file_ = main_path + 'modles/' + file_name
#             os.remove(file_)
#             print('Removed Successful! -- ', file_name)
  print("**********checkpoint_loaded: ", epoch_last)
  initial_epoch = int(epoch_last.split('-')[2]) - 1
  print('**********Begin from epoch: ', str(initial_epoch))
    

model.fit_generator(generate_train(batch_size=100), 
                    steps_per_epoch=900, # (total samples) / batch_size 90000/100 = 900
                    epochs=100, 
                    verbose=1, 
                    callbacks=callbacks_list, 
                    validation_data=generate_test(batch_size=100), 
                    validation_steps=100, # 10000/100 = 100
                    class_weight=None, 
                    max_queue_size=10, 
                    workers=1, 
                    use_multiprocessing=False, 
                    shuffle=False, 
                    initial_epoch=initial_epoch
                    )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100

*********************************generate_test()*********************************
*********************************generate_train()*********************************

900/900 [==============================] - 889s 988ms/step - loss: 1.9234 - acc: 0.1423 - val_loss: 1.8620 - val_acc: 0.1533

Epoch 00001: saving model to /content/drive/My Drive/Colab Notebooks/modles/W2--001-1.9234-bigger.hdf5
Epoch 2/100
658/900 [====================>.........] - ETA: 3:28 - loss: 1.4919 - acc: 0.0689

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [19]:
!pwd
!ls "/content/drive/My Drive/Colab Notebooks/modles/"

/content
W2--001-2.0162-bigger.hdf5  W2--007-1.9841-bigger.hdf5
W2--002-2.0124-bigger.hdf5  W2--008-1.9831-bigger.hdf5
W2--003-1.9936-bigger.hdf5  W2--009-2.1618-bigger.hdf5
W2--004-1.9889-bigger.hdf5  W2--010-2.1511-bigger.hdf5
W2--005-1.9830-bigger.hdf5  W2--011-2.1385-bigger.hdf5
W2--006-1.9849-bigger.hdf5  W2--012-2.4226-bigger.hdf5
